In [2]:
from __future__ import division, print_function, absolute_import

import os
from timeit import time
import warnings
import sys
import cv2
import numpy as np
from PIL import Image
from datetime import datetime
from datetime import timedelta
from darknet import darknet
import csv
from deep_sort import preprocessing
from deep_sort import nn_matching
from deep_sort.detection import Detection
from deep_sort.tracker import Tracker
from tools import generate_detections as gdet
from deep_sort.detection import Detection as ddet
from misc.color import get_rgb
from misc.utils import blend_transparent, area_inside
from shapely.geometry import Polygon, box
warnings.filterwarnings('ignore')


/usr/local/lib/python3.5/dist-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"
netMain = None
metaMain = None
altNames =['AdultBird', 'Chick','Egg']
dw=1920.0/416.0
dh=1080.0/416.0

In [4]:
def  convert(detections,dw,dh ):
    boxs=[]
    class_ids=[]
    for detection in detections:
        
        x, y, w, h = detection[2][0],\
            detection[2][1],\
            detection[2][2],\
            detection[2][3]
        boxs.append([(x-w/2)*dw,(y-h/2)*dh,w*dw,h*dh])
        class_ids.append(altNames.index(detection[0].decode("utf-8") ))
    return (boxs, class_ids)

In [5]:
def main():

    global metaMain, netMain, altNames
    configPath = "darknet/cfg/yolov3-seabird.cfg"
    weightPath = "darknet/backup_608/yolov3-seabird_final.weights"
    metaPath = "darknet/cfg/seabird.data"
    if not os.path.exists(configPath):
        raise ValueError("Invalid config path `" +
                         os.path.abspath(configPath)+"`")
    if not os.path.exists(weightPath):
        raise ValueError("Invalid weight path `" +
                         os.path.abspath(weightPath)+"`")
    if not os.path.exists(metaPath):
        raise ValueError("Invalid data file path `" +
                         os.path.abspath(metaPath)+"`")
    if netMain is None:
        netMain = darknet.load_net_custom(configPath.encode(
            "ascii"), weightPath.encode("ascii"), 0, 1)  # batch size = 1
    if metaMain is None:
        metaMain = darknet.load_meta(metaPath.encode("ascii"))
    if altNames is None:
        try:
            with open(metaPath) as metaFH:
                metaContents = metaFH.read()
                import re
                match = re.search("names *= *(.*)$", metaContents,
                                  re.IGNORECASE | re.MULTILINE)
                if match:
                    result = match.group(1)
                else:
                    result = None
                try:
                    if os.path.exists(result):
                        with open(result) as namesFH:
                            namesList = namesFH.read().strip().split("\n")
                            altNames = [x.strip() for x in namesList]
                except TypeError:
                    pass
        except Exception:
            pass


   # Definition of the parameters
    max_cosine_distance = 0.3
    nn_budget = None
    nms_max_overlap = 1
   # deep_sort 
    model_filename = 'model_data/mars-small128.pb'
    encoder = gdet.create_box_encoder(model_filename,batch_size=3)
    
    metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
    tracker = Tracker(metric)

    writeVideo_flag = True
    
    #video_capture = cv2.VideoCapture('/data/Farallon3_20190706_000001_No001.avi')
    #video_capture = cv2.VideoCapture('/data/15_fps/Farallon3_20190620_021546_No001.mp4')
    video_capture = cv2.VideoCapture('/data/15_fps/Farallon3_20190621_090300_No004.mp4')
    #video_capture = cv2.VideoCapture('f.avi')
    #video_capture = cv2.VideoCapture('/data/fight.avi')

    video_fps = video_capture.get(cv2.CAP_PROP_FPS)
    #video_fps = 25
    list_file = open('events.csv', 'w')
    track_log_file = open('track_log.csv', 'w')
    wr = csv.writer(list_file, dialect='excel')
    wr_tracklog = csv.writer(track_log_file, dialect='excel')
    if writeVideo_flag:
    # Define the codec and create VideoWriter object
        w = int(video_capture.get(3))
        h = int(video_capture.get(4))
        fourcc = cv2.VideoWriter_fourcc(*'MJPG')
        out = cv2.VideoWriter('checkp.avi', fourcc, 15, (w, h))
        zone = cv2.imread('mask/test_zone.png', -1)
     # Create an image we reuse for each detect
    darknet_image = darknet.make_image(darknet.network_width(netMain),
                                    darknet.network_height(netMain),3)
    
    fps = 0.0
    frame_index = -1 
    mask = cv2.imread('mask/mask_new.jpg')
    mask = np.uint8(mask/255)
    ##############################################
    #points = [(130,102),(535,370),(808,345),(1570,391),(1494,808),(373,817),(4,496),(1,276)]
    #points = [(4,22),(121,96),(207,99),(537,366),(819,324),(1564,385),(1764,322),(1648,889),(105,915),(3,762)]
    points = [(2,24),(127,24),(579,321),(1746,336),(1674,878),(1912,957),(1926,1081),(2,1074)]
    zone_polygon = Polygon(points)
    text_count_size = 9e-3 * 200
    text_count_x,text_count_y = 550,  1000
    avg_area_box_all_frames = 0
    time_stamp = datetime(2019, 6, 21, 10, 33, 5) 
    
    while True:
        ret, frame = video_capture.read()  # frame shape 640*480*3
        if ret != True:
            break
        frame_index = frame_index +1  
        t1 = time.time()
        write = 0 
        if frame_index % 15 == 0:
           write = 1 
            
        draw_frame  = frame 
        frame = np.multiply(frame, mask)
        # image = Image.fromarray(frame)
        #image = Image.fromarray(frame[...,::-1]) #bgr to rgb
        
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_resized = cv2.resize(frame_rgb,
                                   (darknet.network_width(netMain),
                                    darknet.network_height(netMain)),
                                   interpolation=cv2.INTER_LINEAR)
        darknet.copy_image_from_bytes(darknet_image,frame_resized.tobytes())
        detections = darknet.detect_image(netMain, metaMain, darknet_image, thresh=0.5)

        boxs, class_ids=convert(detections, dw, dh)
        features = encoder(frame,boxs)
        # features = [[] for i in range(len(boxs))]
        
        # score to 1.0 here).
        dets = [Detection(bbox, 1.0, feature) for bbox, feature in zip(boxs, features)]
        
        # Run non-maxima suppression.
        boxes = np.array([d.tlwh for d in dets])
        scores = np.array([d.confidence for d in dets])
        indices = preprocessing.non_max_suppression(boxes, nms_max_overlap, scores)
        detections_tracker = [dets[i] for i in indices]
        ids = [class_ids[i] for i in indices]
        time_lapsed_sec = frame_index / video_fps
        time_stamp_now = time_stamp + timedelta(seconds=time_lapsed_sec) 

        # Call the tracker
        tracker.predict()
        tracker.update(detections_tracker, ids, time_stamp_now)
        tracker.update_events(draw_frame, time_stamp_now, wr, wr_tracklog, write )
        tracker.update_terr()
        if writeVideo_flag: 
             k = tracker.terr
             for p  in k.keys() :
                 cv2.circle(draw_frame,(int(k[p].x),int(k[p].y)),100,(0,0,255),thickness=2,lineType=8,shift=0)
   
            #   draw_frame = blend_transparent(draw_frame, zone)
            #cv2.rectangle(draw_frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])),(255,255,255), 2)
        avg_area_box = 0
        for det in detections:
            name , x, y, w, h = det[0],det[2][0],det[2][1],det[2][2],det[2][3]
            class_id = altNames.index(name.decode("utf-8") )
            bbox = [(x-w/2)*dw,(y-h/2)*dh,(x+w/2)*dw,(y+h/2)*dh]
            area_box = w*h*dw*dh
            if area_box > 3*avg_area_box_all_frames:
                 cv2.rectangle(draw_frame,(int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])),(255,255,255), 3)
                 cv2.putText(draw_frame,"Flapping detected !!!!",(text_count_x,text_count_y),4,text_count_size, (255,0,0),4) 
            avg_area_box = avg_area_box + area_box
            cv2.rectangle(draw_frame,(int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])),get_rgb(class_id,3), 2)
        avg_area_box = avg_area_box/len(detections)
        avg_area_box_all_frames=avg_area_box 
        ##cv2.imshow('', frame)
        
        if writeVideo_flag:
            # save a frame
            out.write(draw_frame)
      
        fps  = ( fps + (1./(time.time()-t1)) ) / 2
        print("fps= %f"%(fps))
        
        # Press Q to stop!
        #if cv2.waitKey(1) & 0xFF == ord('q'):
        #    break

    video_capture.release()
    if writeVideo_flag:
        out.release()
        list_file.close()
    cv2.destroyAllWindows()

In [6]:
if __name__ == '__main__':
    main()


fps= 0.627714
fps= 8.706661
fps= 12.383043
fps= 13.966757
fps= 13.120714
fps= 11.755095
fps= 11.306463
fps= 11.222483
fps= 10.880230
fps= 11.132339
fps= 11.189933
fps= 10.695553
fps= 11.806176
fps= 12.739212
fps= 13.247160
fps= 13.108481
fps= 13.438703
fps= 13.591902
fps= 13.640531
fps= 13.710949
fps= 13.761680
fps= 13.677395
fps= 13.555086
fps= 13.435168
fps= 13.396838
fps= 13.326125
fps= 13.370536
fps= 13.362756
fps= 11.994955
fps= 11.469727
fps= 10.717974
fps= 10.564491
fps= 10.446106
fps= 10.267691
fps= 10.600033
fps= 11.289757
fps= 12.182074
fps= 12.526309
fps= 11.397972
fps= 11.119510
fps= 10.553474
fps= 10.954567
fps= 11.332600
fps= 11.315681
fps= 11.394771
fps= 11.414157
fps= 11.339799
fps= 11.979951
fps= 12.629495
fps= 12.960053
fps= 13.147621
fps= 13.203591
fps= 13.202861
fps= 13.362686
fps= 13.479992
fps= 13.365378
fps= 13.315992
fps= 13.382525
fps= 13.293887
fps= 13.300155
fps= 13.105514
fps= 12.849717
fps= 11.607951
fps= 10.961841
fps= 10.604448
fps= 10.874066
fps= 10.9612

KeyboardInterrupt: 

In [ ]:
import io

import base64

from IPython.display import HTML

video = io.open('cat.mp4', 'r+b').read()

encoded = base64.b64encode(video)

HTML(data='''<video alt="test" controls>

<source src="data:video/mp4;base64,{0}" type="video/mp4" /> </video>'''.format(encoded.decode('ascii')))